### Sentimental_analysis

#### 1. 데이터 읽어오기

In [ ]:
import numpy as np
import keras
from  keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import os
import pandas as pd
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/sentimental_analysis_project/data/train.txt', header =None, sep =';', names = ['Sentence','Emotion'], encoding='utf-8')
df_test = pd.read_csv('/content/drive/MyDrive/sentimental_analysis_project/data/test.txt', header = None, sep =';', names = ['Sentence','Emotion'],encoding='utf-8')
df_val=pd.read_csv('/content/drive/MyDrive/sentimental_analysis_project/data/val.txt',header=None,sep=';',names=['Sentence','Emotion'],encoding='utf-8')

In [ ]:
df_train.info()

#### 2. train_set 데이터 분포와 특징 확인

In [ ]:
df_train['Emotion'].value_counts()

In [ ]:
# 막대 그래프 그려 확인하기
# 전체 값의 10%의 적은 비중을 차지하는 surprise 같은 경우 불균형 데이터라고 여겨질 수 있어 눈여겨 보아야 한다. 
value=df_train['Emotion'].value_counts()
plt.bar(value.index, value.values)

#### 3. wordcloud로 분포 확인하기

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
from nltk.tokenize import RegexpTokenizer
import re

In [ ]:
# 빈도수가 높은 단어를 출력하여 시각화한다.

In [ ]:
# 공백 문자 기준 잘라 소문자로 모두 변환하여 리스트 형식으로 담는다.
tokens = re.findall("[\w']+", df_train['Sentence'].str.lower().str.cat(sep=' ')) 

In [ ]:
# nltk의 stopwords를 가져와 tokens 변수에서 stopwords에 해당되지 않는 단어를 걸러 filtered_sentence에 저장한다.
nltk.download('stopwords')
en_stops = set(stopwords.words('english'))

filtered_sentence = []
filtered_sentence=[word for word in tokens if word not in en_stops]

In [ ]:
text = list(set(filtered_sentence)) # 중복 제거

# 전체데이터의 wordcloud 출력
plt.rcParams['figure.figsize'] = (16, 16)
wordcloud = WordCloud(max_font_size=50, max_words=100,background_color="white").generate(' '.join(text))
plt.imshow(wordcloud,interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# 하나의 감정분류(sadness, anger, etc.)의 wordcloud를 보고 싶다면 아래와 전처리하고 filtered_sentence에 담아 출력한다.

In [ ]:
# fear에 해당하는 데이터만 갖고 오기
tokens = re.findall("[\w']+", df_train[df_train['Emotion']=='fear']['Sentence'].str.lower().str.cat(sep=' '))
filtered_sentence = [token for token in tokens if not token in en_stops]
filtered_sentence

#### 4. validation_set과 test_set data도 확인

In [ ]:
df_val.info()

In [ ]:
df_test.info()

#### 5. 단어의 토큰화

In [ ]:
# 단어 혹은 글자 단위로 토큰화(tokenize)
nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
token_sentence = []
for sentence in df_train['Sentence']:
    token_sentence.append(word_tokenize(sentence))
print(token_sentence)

df_train['token_sentence'] = token_sentence
df_train

#### 6. stop_words 제거

In [ ]:
filtered_sentence = []
stop_words = set(stopwords.words('english'))

for sentence in df_train['token_sentence']:
    word_token=[word for word in sentence if word not in stop_words]
    filtered_sentence.append(word_token)
print(len(filtered_sentence))

#### 7. 표준화(Stemming) 
#### 감성 분석의 경우 시제(과거-현재-미래)에 구분 없이 뜻에 포커스 할 수 있도록 표준화하는 작업이 필요하다.

In [ ]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [ ]:
stemming_sentence = []
for sentence in filtered_sentence:
    word_token = [stemmer.stem(word) for word in sentence]
    stemming_sentence.append(word_token)

#### 8. Bag of words(BOW) 만들기

In [ ]:
from  keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(oov_token='UNK')
tokenizer.fit_on_texts(stemming_sentence)

In [ ]:
print(len(tokenizer.word_index)) # 10,376 rows
tokenizer.word_index 

#### 9. Padding
#### 토큰을 인덱스 값으로 변환

In [ ]:
# sequence로 변경
# 문장의 길이가 모두 다르지만 딥러닝은 입력값을 고정값으로 받기 때문에 padding으로 최대 길이에 맞게 맞춰준다.
X_train=tokenizer.texts_to_sequences(stemming_sentence)
X_train

In [ ]:
# 문장 길이 분포를 그래프로 확인하기
import matplotlib.pyplot as plt

num_token_per_sentence = [ len(sentence) for sentence in X_train ]
# print(max(num_token_per_sentence))
plt.hist(num_token_per_sentence)
plt.show()

In [ ]:
# pad_sequences를 사용하여 X_train 길이 통일하기
X_train_pad=pad_sequences(X_train)
X_train_pad

#### 10. Label 데이터 변환

#### `'joy':0,'anger':1,'love':2,'sadness':3,'fear':4,'surprise':5` 방식으로 맵핑, 변환한다.

In [ ]:
df_train['Emotion']=df_train['Emotion'].replace({'joy':0,'anger':1,'love':2,'sadness':3,'fear':4,'surprise':5})
df_train['Emotion']
y_train = np.array(df_train['Emotion'].values)
y_train

#### 11. validation, test dataset 데이터도 동일하기 전처리

In [ ]:
stemming_sentence_val = []
stemming_sentence_test = []

for sentence in df_val['Sentence']:
    word_token=word_tokenize(sentence)
    word_token=[stemmer.stem(word) for word in word_token]
    stemming_sentence_val.append(word_token)
    
for sentence in df_test['Sentence']:
    word_token=word_tokenize(sentence)
    word_token=[stemmer.stem(word) for word in word_token]
    stemming_sentence_test.append(word_token)
 
X_val=tokenizer.texts_to_sequences(stemming_sentence_val)
X_test=tokenizer.texts_to_sequences(stemming_sentence_test)

X_val_pad=pad_sequences(X_val)
X_test_pad=pad_sequences(X_test)

df_val['Emotion']=df_val['Emotion'].replace({'joy':0,'anger':1,'love':2,'sadness':3,'fear':4,'surprise':5})
df_test['Emotion']=df_test['Emotion'].replace({'joy':0,'anger':1,'love':2,'sadness':3,'fear':4,'surprise':5})

y_val = np.array(df_val['Emotion'].values)
y_test = np.array(df_test['Emotion'].values)

#### 12. 딥러닝 모델 설정
#### 감성 분류를 위해 LSTM을 사용한다.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, SimpleRNN, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

In [ ]:
sequence_length = 35 # max(num_token_per_sentence)
vocabulary_size = len(tokenizer.word_index)+1 # 10376
embedding_dim = 256

In [ ]:
model = Sequential([
    Embedding(vocabulary_size, embedding_dim),
    LSTM(units=50, input_shape=(vocabulary_size, embedding_dim), return_sequences=True),
    LSTM(units=25),
    Dense(20, activation="relu"),
    Dropout(0.05),
    Dense(6, activation = "softmax")
    ])

# model compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']
)

#### 13. 학습
#### epoch = 10, batch_size = 64

In [ ]:
history=model.fit(X_train_pad, y_train, batch_size=64, epochs=10, validation_data=(X_val_pad, y_val))

#### 14. 평가 및 예측

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test)

print('test set accuracy: ', test_accuracy)

In [ ]:
test_prediction =model.predict(X_test_pad)
test_prediction

In [ ]:
for i in range(5): 
    test_prediction
    print("{} 번째 테스트 데이터 문장: \n{}".format(i, df_test['Sentence'][i]))
    print("{} 번째 테스트 데이터의 예측 결과: {}, 실제 데이터: {}\n".format(i, test_prediction[i], df_test['Emotion'][i]))